In [1]:
import numpy as np
import os

In [2]:
fname = "/Users/samirabaghbanbari/samira/code/POR-Water-pos-1.xyz"

fyle = open(fname, "r")
lines = fyle.readlines()
fyle.close()

n_step = 800
len_traj = 809

line_idx_starts, line_idx_ends = [], []
for line_idx, line in enumerate(lines[:]):
    if "i =" in line:
        line_idx_starts.append(line_idx + 1)
        line_idx_ends.append(line_idx + len_traj)

line_idx_starts_sub = line_idx_starts[::n_step]
line_idx_ends_sub = line_idx_ends[::n_step]

trajs = []
for st_idx, end_idx in zip(line_idx_starts_sub, line_idx_ends_sub):
    atoms, X, Y, Z = [], [], [], []
    for sub_line in lines[st_idx: end_idx + 1]:
        atom, x, y, z = sub_line.strip().split("      ")
        atoms.append(atom)
        X.append(float(x))
        Y.append(float(y))
        Z.append(float(z))
        single_traj = np.array([atoms, X, Y, Z], dtype="U14").T
    
    trajs.append(single_traj)

trajs = np.array(trajs)
np.save("output.npy", trajs)



# Convert and save to .xyz format
with open("output.xyz", "w") as xyz_file:
    for frame in trajs:
        n_atoms = frame.shape[0]
        xyz_file.write(f"{n_atoms}\n")
        xyz_file.write("Extracted frame\n")
        for row in frame:
            atom = row[0]
            x = float(row[1])
            y = float(row[2])
            z = float(row[3])
            xyz_file.write(f"{atom} {x:.6f} {y:.6f} {z:.6f}\n")




In [3]:
import numpy as np

# Step 1: Load from original .xyz file
with open("output.xyz", "r") as f:
    lines = f.readlines()

frames = []
i = 0
while i < len(lines):
    n_atoms = int(lines[i].strip())
    comment = lines[i + 1].strip()
    atoms = []
    for j in range(i + 2, i + 2 + n_atoms):
        parts = lines[j].split()
        atom = parts[0]
        x, y, z = map(float, parts[1:4])
        atoms.append([atom, x, y, z])
    frames.append(atoms)
    i += 2 + n_atoms

frames = np.array(frames, dtype=object)  # shape: (n_frames, n_atoms, 4)

# Step 2: Save as .npy for convenience
np.save("output.npy", frames)

# Step 3: Load .npy and crop last 41 atoms from each frame
my_data = np.load("output.npy", allow_pickle=True)
my_data_sub = my_data[:, -41:, :]
np.save("cropped_porphyrine.npy", my_data_sub)

# Step 4: Save cropped fragments as .xyz file(s)
# Option 1: Save all into one file
#with open("cropped_porphyrine.xyz", "w") as f:
#    for frame in my_data_sub:
#        f.write(f"{len(frame)}\n")
#        f.write("Cropped porphyrin fragment\n")
#       for atom in frame:
#            f.write(f"{atom[0]} {float(atom[1]):.6f} {float(atom[2]):.6f} {float(atom[3]):.6f}\n")

# Option 2: Save each frame as a separate file
for i, frame in enumerate(my_data_sub):
      with open(f"fragment_{i:04d}.xyz", "w") as f:
          f.write(f"{len(frame)}\n")
          f.write(f"Cropped frame {i}\n")
          for atom in frame:
              f.write(f"{atom[0]} {float(atom[1]):.6f} {float(atom[2]):.6f} {float(atom[3]):.6f}\n")


In [4]:
my_data = np.load("output.npy", allow_pickle=True)
my_data_sub = my_data[:, -41:, :]
np.save("cropped_porphyrine.npy", my_data_sub)

splitting output.npy to 50 xyz files

In [5]:
my_data = np.load("output.npy", allow_pickle=True)
output_dir = "/Users/samirabaghbanbari/samira/code/output_50_xyz"
os.makedirs(output_dir, exist_ok=True)

for i in range(50):
    frame = my_data[i]
    filename = os.path.join(output_dir, f"frame_{i+1}.txt")

    with open(filename, 'w') as f:
        f.write("atom x y z\n")
        for row in frame:
            atom, x, y, z = row
            f.write(f"{atom} {x:.6f} {y:.6f} {z:.6f}\n")